In [1]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
pd.options.display.float_format = '{:,.3f}'.format
import argparse
pd.options.mode.chained_assignment = None
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale = 1.5)
sns.set_context(rc={"lines.markersize": 10}) # controls size of style markers in line plots

import matplotlib

import pickle as pkl
from matplotlib import pyplot as plt
new_plot_col=list(range(1800,2010,20))
from scipy.stats.stats import pearsonr
from scipy.stats.stats import pearsonr

from functools import reduce

In [2]:
br_to_us=pd.read_excel("../data/Book.xlsx",skiprows=[0])
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
spelling_replacement={'modifier':br_to_us_dict,'head':br_to_us_dict}

def lemma_maker(x, y):
    #print(lemmatizer.lemmatize(x,y))
    return lemmatizer.lemmatize(x,y)

In [3]:
parser = argparse.ArgumentParser(description='Compute features from embeddings')

parser.add_argument('--temporal',  type=int,
                    help='Value to bin the temporal information: 0 (remove temporal information), 1 (no binning), 10 (binning to decades), 20 (binning each 20 years) or 50 (binning each 50 years)')

parser.add_argument('--cutoff', type=int, default=50,
                    help='Cut-off frequency for each compound per time period : none (0), 20, 50 and 100')

parser.add_argument('--contextual', action='store_true',
                    help='Is the model contextual')

args = parser.parse_args('--temporal 20 --cutoff 20'.split())

In [4]:
print(f'Cutoff: {args.cutoff}')
print(f'Time span:  {args.temporal}')

Cutoff: 20
Time span:  20


In [5]:
temp_cutoff_str=str(args.temporal)+'_'+str(args.cutoff)

In [24]:
if args.contextual:
    comp_df_path='../../datasets/compounds_CompoundAware_'+temp_cutoff_str+'_300.pkl'
    mod_df_path='../../datasets/modifiers_CompoundAware_'+temp_cutoff_str+'_300.pkl'
    head_df_path='../../datasets/heads_CompoundAware_'+temp_cutoff_str+'_300.pkl'
    features_df_path='../../datasets/features_CompoundAware_'+temp_cutoff_str+'_300.pkl'
else:
    comp_df_path='../../datasets/compounds_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'
    mod_df_path='../../datasets/constituents_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'
    head_df_path='../../datasets/constituents_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'
    features_df_path='../../datasets/features_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'

In [7]:
heads=pd.read_pickle(head_df_path)

if args.temporal!=0:
    heads.index.set_names('time', level=1,inplace=True)
    heads.index.set_names('head',level=0,inplace=True)

In [8]:
modifiers=pd.read_pickle(mod_df_path)

if args.temporal!=0:
    modifiers.index.set_names('time', level=1,inplace=True)
    modifiers.index.set_names('modifier',level=0,inplace=True)

In [9]:
compounds=pd.read_pickle(comp_df_path)

if args.temporal!=0:
    compounds.index.set_names('time', level=2,inplace=True)
compounds.drop(['common'],axis=1,inplace=True)
compounds=compounds+1

In [10]:
if args.temporal!=0:
    
    compound_decade_counts=compounds.groupby('time').sum().sum(axis=1).to_frame()
    compound_decade_counts.columns=['N']

#compounds = dd.from_pandas(compounds, npartitions=30)

    XY=compounds.groupby(['modifier','head','time']).sum().sum(axis=1).to_frame()
    X_star=compounds.groupby(['modifier','time']).sum().sum(axis=1).to_frame()
    Y_star=compounds.groupby(['head','time']).sum().sum(axis=1).to_frame()


else:
    XY=compounds.groupby(['modifier','head']).sum().sum(axis=1).to_frame()
    X_star=compounds.groupby(['modifier']).sum().sum(axis=1).to_frame()
    Y_star=compounds.groupby(['head']).sum().sum(axis=1).to_frame()


    
    
#XY=XY.compute()
XY.columns=['a']

#X_star=X_star.compute()
X_star.columns=['x_star']
#Y_star=Y_star.compute()
Y_star.columns=['star_y']


if args.temporal!=0:
 
    merge1=pd.merge(XY.reset_index(),X_star.reset_index(),on=['modifier','time'])

    information_feat=pd.merge(merge1,Y_star.reset_index(),on=['head','time'])
else:
    merge1=pd.merge(XY.reset_index(),X_star.reset_index(),on=['modifier'])

    information_feat=pd.merge(merge1,Y_star.reset_index(),on=['head'])    

#information_feat=dd.from_pandas(information_feat, npartitions=30)
information_feat['b']=information_feat['x_star']-information_feat['a']
information_feat['c']=information_feat['star_y']-information_feat['a']

if args.temporal!=0:
    information_feat=pd.merge(information_feat,compound_decade_counts.reset_index(),on=['time'])



#information_feat=information_feat.compute()
else:
 
    information_feat['N']=compounds.reset_index().drop(['modifier','head'],axis=1).sum().sum()
    

#information_feat=dd.from_pandas(information_feat, npartitions=30)
information_feat['d']=information_feat['N']-(information_feat['a']+information_feat['b']+information_feat['c'])
information_feat['x_bar_star']=information_feat['N']-information_feat['x_star']
information_feat['star_y_bar']=information_feat['N']-information_feat['star_y']
#information_feat['LR']=-2*np.sum(information_feat['a']*np.log2((information_feat['a']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y'])))

#information_feat=information_feat.compute()
if args.temporal!=0:

    information_feat.set_index(['modifier','head','time'],inplace=True)
else:
    information_feat.set_index(['modifier','head'],inplace=True)


information_feat.replace(0,0.0001,inplace=True)
information_feat['log_ratio']=2*(information_feat['a']*np.log((information_feat['a']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y']))+\
information_feat['b']*np.log((information_feat['b']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y_bar']))+\
information_feat['c']*np.log((information_feat['c']*information_feat['N'])/(information_feat['x_bar_star']*information_feat['star_y']))+\
information_feat['d']*np.log((information_feat['d']*information_feat['N'])/(information_feat['x_bar_star']*information_feat['star_y_bar'])))
information_feat['ppmi']=np.log2((information_feat['a']*information_feat['N'])/(information_feat['x_star']*information_feat['star_y']))
information_feat['local_mi']=information_feat['a']*information_feat['ppmi']
information_feat.ppmi.loc[information_feat.ppmi<=0]=0
information_feat.drop(['a','x_star','star_y','b','c','d','N','d','x_bar_star','star_y_bar'],axis=1,inplace=True)
#information_feat.info()
#information_feat.head()

In [13]:
compounds=compounds-1


compound_modifier_sim=compounds.multiply(modifiers.reindex(compounds.index, method='ffill')).sum(axis=1).to_frame()
compound_modifier_sim.columns=['sim_with_modifier']


compound_head_sim=compounds.multiply(heads.reindex(compounds.index, method='ffill')).sum(axis=1).to_frame()
compound_head_sim.columns=['sim_with_head']

prod_mod=compound_modifier_sim.groupby('modifier').size().to_frame()
prod_mod.columns=['modifier_prod']

prod_head=compound_modifier_sim.groupby('head').size().to_frame()
prod_head.columns=['head_prod']

if args.temporal!=0:
    constituent_sim=compounds.reset_index()[['modifier','head','time']].merge(modifiers.reset_index(),how='left',on=['modifier','time'])
    constituent_sim.set_index(['modifier','head','time'],inplace=True)
else:
    constituent_sim=compounds.reset_index()[['modifier','head']].merge(modifiers.reset_index(),how='left',on=['modifier'])
    constituent_sim.set_index(['modifier','head'],inplace=True)


constituent_sim=constituent_sim.multiply(heads.reindex(constituent_sim.index, method='ffill')).sum(axis=1).to_frame()
constituent_sim.columns=['sim_bw_constituents']

dfs = [constituent_sim, compound_head_sim, compound_modifier_sim, information_feat]
compounds_final = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True), dfs)
compounds_final=pd.merge(compounds_final.reset_index(),prod_mod.reset_index(),how='left',on=['modifier'])
compounds_final=pd.merge(compounds_final,prod_head.reset_index(),how='left',on=['head'])


In [14]:
if args.temporal!=0:
    compounds_final=pd.pivot_table(compounds_final.reset_index(), index=['modifier','head'], columns=['time'])

    compounds_final.fillna(0,inplace=True)
    compounds_final -= compounds_final.min()
    compounds_final /= compounds_final.max()
    compounds_final_1=compounds_final.columns.get_level_values(0)
    compounds_final_2=compounds_final.columns.get_level_values(1)

    cur_year=0
    new_columns=[]
    for year in compounds_final_2:
        new_columns.append(str(year)+"_"+compounds_final_1[cur_year])
        cur_year+=1
    compounds_final.columns=new_columns


else:
    #compounds_final = reduce(lambda left,right: pd.merge(left,right,on=['modifier','head']), dfs)
    #compounds_final.drop(['head_denom','modifier_denom'],axis=1,inplace=True)
    compounds_final.set_index(['modifier','head'],inplace=True)
    compounds_final.fillna(0,inplace=True)
    compounds_final -= compounds_final.min()
    compounds_final /= compounds_final.max()

In [17]:
reddy_comp=pd.read_csv("../data/reddy_compounds.txt",sep="\t")
#print(reddy_comp.columns)
reddy_comp.columns=['compound','to_divide']
reddy_comp['modifier_mean'],reddy_comp['modifier_std'],reddy_comp['head_mean'],reddy_comp['head_std'],reddy_comp['compound_mean'],reddy_comp['compound_std'],_=reddy_comp.to_divide.str.split(" ",7).str
reddy_comp['modifier'],reddy_comp['head']=reddy_comp['compound'].str.split(" ",2).str
reddy_comp.modifier=reddy_comp.modifier.str[:-2]
reddy_comp['head']=reddy_comp['head'].str[:-2]
reddy_comp.drop(['compound','to_divide'],axis=1,inplace=True)
reddy_comp['modifier']=np.vectorize(lemma_maker)(reddy_comp['modifier'],'n')
reddy_comp['head']=np.vectorize(lemma_maker)(reddy_comp['head'],'n')
reddy_comp.replace(spelling_replacement,inplace=True)
#reddy_comp['modifier']=reddy_comp['modifier']+"_noun"
#reddy_comp['head']=reddy_comp['head']+"_noun"
reddy_comp=reddy_comp.apply(pd.to_numeric, errors='ignore')
#reddy_comp.set_index(['modifier','head'],inplace=True)

In [21]:
comp_90=pd.read_csv("../data/compounds90.txt",sep="\t")
comp_90['mod_pos'],comp_90['head_pos']=comp_90.compound_lemmapos.str.split('_').str
comp_90['modifier'],comp_90['mod_pos']=comp_90.mod_pos.str.split('/').str
comp_90['head'],comp_90['head_pos']=comp_90.head_pos.str.split('/').str
comp_90=comp_90.loc[~(comp_90.mod_pos=="ADJ")]
comp_90=comp_90.loc[:,['avgModifier','stdevModifier','avgHead','stdevHeadModifier','compositionality','stdevHeadModifier','modifier','head']]
comp_90.columns=reddy_comp.columns

,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,modifier,head
1,0.833,0.985,4.556,1.283,1.333,1.283,armchair,critic
2,4.500,1.043,3.667,1.392,3.944,1.392,baby,buggy
6,4.700,0.733,4.650,0.598,4.600,0.598,birth,rate
8,4.250,1.238,4.688,1.238,4.250,1.238,bow,tie
9,3.353,1.539,2.294,1.320,2.647,1.320,brain,teaser
11,0.706,1.105,4.941,1.200,1.235,1.200,carpet,bombing
15,4.667,0.840,4.556,0.985,4.500,0.985,computer,program
16,4.500,1.000,1.250,1.447,2.100,1.447,con,artist
17,4.684,0.671,4.632,0.582,4.684,0.582,cooking,stove
18,1.105,1.150,4.316,1.134,1.790,1.134,cotton,candy


In [22]:
comp_ext=pd.read_csv("../data/compounds_ext.txt",sep="\t")
comp_ext['mod_pos'],comp_ext['head_pos']=comp_ext.compound_lemmapos.str.split('_').str
comp_ext['modifier'],comp_ext['mod_pos']=comp_ext.mod_pos.str.split('/').str
comp_ext['head'],comp_ext['head_pos']=comp_ext.head_pos.str.split('/').str
comp_ext=comp_ext.loc[~(comp_ext.mod_pos=="ADJ")]

comp_ext=comp_ext.loc[:,['avgModifier','stdevModifier','avgHead','stdevHeadModifier','compositionality','stdevHeadModifier','modifier','head']]
comp_ext.columns=reddy_comp.columns

In [23]:
all_compounds=pd.concat([reddy_comp,comp_ext,comp_90],ignore_index=True)
all_compounds['modifier']=all_compounds['modifier']+"_noun"
all_compounds['head']=all_compounds['head']+"_noun"
all_compounds

,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,modifier,head
0,3.867,1.118,4.867,0.340,4.250,0.871,end_noun,user_noun
1,1.607,1.655,1.893,1.496,1.704,1.717,firing_noun,line_noun
2,2.821,1.965,4.862,0.345,3.828,1.234,game_noun,plan_noun
3,4.767,0.423,4.862,0.345,4.800,0.476,application_noun,form_noun
4,0.600,0.800,4.586,1.099,1.310,1.021,snail_noun,mail_noun
...,...,...,...,...,...,...,...,...
205,2.684,1.493,4.158,1.344,3.158,1.344,street_noun,girl_noun
206,4.737,0.452,4.421,0.597,4.632,0.597,subway_noun,system_noun
207,2.222,1.800,4.333,1.757,2.500,1.757,tennis_noun,elbow_noun
208,4.100,1.252,0.350,1.395,1.050,1.395,top_noun,dog_noun


In [32]:
compounds_final[['1800_index']]

1800_index
modifier     head                     
'_noun       a'_noun             0.000
             a'isha_noun         0.000
             a'ishah_noun        0.000
             a_noun              0.000
             aaa_noun            0.000
...                                ...
zymogen_noun form_noun           0.000
             granule_noun        0.000
zysman_noun  american_noun       0.000
zz_noun      top_noun            0.000
zzzz_noun    best_noun           0.000

[1192653 rows x 1 columns]

In [27]:
merge_df=all_compounds.merge(compounds_final.reset_index(),on=['modifier','head'],how='inner')
merge_df.set_index(["modifier", "head"], inplace = True)

merge_df.to_csv(features_df_path,sep='\t')